In [1]:
%cd ..

/Users/yuqingwang/Desktop/Time Series Final


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import statsmodels.api as sm
from scipy.optimize import minimize

In [3]:
def load_data(file):
    Stock = pd.read_csv(file)
    Stock['Date'] = pd.to_datetime(Stock['Date'])
    Stock.set_index(['Date'],inplace=True)
    Stock = Stock['Close']
    Stock_return = np.log(Stock/Stock.shift())
    Stock_return.dropna(inplace = True)
    return Stock_return

In [4]:
FB_return = load_data('data/FB.csv')

In [5]:
FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')]

Date
2016-01-05    0.004977
2016-01-06    0.002333
2016-01-07   -0.050287
2016-01-08   -0.006044
2016-01-11    0.001848
                ...   
2019-11-11   -0.006466
2019-11-12    0.025309
2019-11-13   -0.006604
2019-11-14   -0.000207
2019-11-15    0.010045
Name: Close, Length: 975, dtype: float64

In [6]:
FB_return_mean = FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')].mean()
FB_return_mean

0.0006629588874457139

In [7]:
(FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')]-FB_return_mean)**2

Date
2016-01-05    1.860969e-05
2016-01-06    2.790631e-06
2016-01-07    2.595888e-03
2016-01-08    4.497674e-05
2016-01-11    1.403541e-06
                  ...     
2019-11-11    5.082239e-05
2019-11-12    6.074066e-04
2019-11-13    5.280497e-05
2019-11-14    7.570249e-07
2019-11-15    8.802683e-05
Name: Close, Length: 975, dtype: float64

## Constant Volatiltiy

In [8]:
vol = np.std(FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')])
vol

0.018451852944634566

## ARCH(1)

In [9]:
parameter_ARCH = np.array([vol,0.3,FB_return_mean])#alfa_0,alfa_1,miu


def L_ARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[2])*(i-parameter_0[2])/(sigma2)/2
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[2])*(i-parameter_0[2])
    return result

res_ARCH = minimize(L_ARCH, parameter_ARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_ARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2526.485361
         Iterations: 200
         Function evaluations: 373


 final_simplex: (array([[2.62493352e-04, 3.41183517e-01, 6.12495131e-04],
       [2.62493351e-04, 3.41183517e-01, 6.12495146e-04],
       [2.62493351e-04, 3.41183518e-01, 6.12495142e-04],
       [2.62493352e-04, 3.41183517e-01, 6.12495143e-04]]), array([-2526.48536088, -2526.48536088, -2526.48536088, -2526.48536088]))
           fun: -2526.4853608838916
       message: 'Optimization terminated successfully.'
          nfev: 373
           nit: 200
        status: 0
       success: True
             x: array([2.62493352e-04, 3.41183517e-01, 6.12495131e-04])

In [10]:
res_ARCH.x

array([2.62493352e-04, 3.41183517e-01, 6.12495131e-04])

In [11]:
ARCH_estimated_vol = []
sigma2 = vol*vol
for i in FB_return:
    sigma2 = res_ARCH.x[0]+res_ARCH.x[1]*(i-res_ARCH.x[2])*(i-res_ARCH.x[2])
    ARCH_estimated_vol.append(sigma2)


In [12]:
ARCH_estimated_vol.pop
ARCH_estimated_vol = pd.DataFrame(ARCH_estimated_vol, index = FB_return.index, columns = ['ARCH_estimated_vol'])

In [13]:
ARCH_estimated_vol.mean()

ARCH_estimated_vol    0.000377
dtype: float64

In [28]:
#np.sqrt(ARCH_estimated_vol*252).to_pickle('Estimated Volatility/FB_ARCH_estimated_vol.pkl')

## GARCH(1,1)

In [15]:
parameter_GARCH = np.array([vol,0.34,0.66,FB_return_mean])#alfa_0,alfa_1,beta_1,miu


def L_GARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[3])*(i-parameter_0[3])/(sigma2)/2
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[2]*sigma2
    return result

res_GARCH = minimize(L_GARCH, parameter_GARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_GARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2545.813215
         Iterations: 492
         Function evaluations: 865


 final_simplex: (array([[8.62736701e-05, 3.54061499e-01, 4.93056131e-01, 2.38418160e-04],
       [8.62736702e-05, 3.54061499e-01, 4.93056130e-01, 2.38418166e-04],
       [8.62736703e-05, 3.54061499e-01, 4.93056131e-01, 2.38418167e-04],
       [8.62736702e-05, 3.54061499e-01, 4.93056130e-01, 2.38418165e-04],
       [8.62736703e-05, 3.54061499e-01, 4.93056130e-01, 2.38418165e-04]]), array([-2545.81321454, -2545.81321454, -2545.81321454, -2545.81321454,
       -2545.81321454]))
           fun: -2545.8132145379714
       message: 'Optimization terminated successfully.'
          nfev: 865
           nit: 492
        status: 0
       success: True
             x: array([8.62736701e-05, 3.54061499e-01, 4.93056131e-01, 2.38418160e-04])

In [16]:
GARCH_estimated_vol = []
sigma2 = vol*vol
for i in FB_return:
    sigma2 = res_GARCH.x[0]+res_GARCH.x[1]*(i-res_GARCH.x[3])*(i-res_GARCH.x[3])+res_GARCH.x[2]*sigma2
    GARCH_estimated_vol.append(sigma2)
GARCH_estimated_vol.pop
GARCH_estimated_vol = pd.DataFrame(GARCH_estimated_vol, index = FB_return.index, columns = ['GARCH_estimated_vol'])
GARCH_estimated_vol.mean()

GARCH_estimated_vol    0.000404
dtype: float64

In [29]:
#np.sqrt(GARCH_estimated_vol*252).to_pickle('Estimated Volatility/FB_GARCH_estimated_vol.pkl')

## TGARCH

In [18]:
parameter_TGARCH = np.array([0,0.35,0.5,0,0])#alfa_0,alfa_1,beta_!,miu,gamma1

def L_TGARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[3])*(i-parameter_0[3])/(sigma2)/2
        if i-parameter_0[3]<0:
            S = 1
        else:
            S = 0
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[4]*S*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[2]*sigma2
    return result

res_TGARCH = minimize(L_TGARCH, parameter_TGARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_TGARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys


Optimization terminated successfully.
         Current function value: -2546.547431
         Iterations: 471
         Function evaluations: 799


 final_simplex: (array([[1.00657587e-04, 2.68078175e-01, 4.54879076e-01, 5.19384295e-05,
        1.54648293e-01],
       [1.00657587e-04, 2.68078175e-01, 4.54879075e-01, 5.19384267e-05,
        1.54648293e-01],
       [1.00657587e-04, 2.68078175e-01, 4.54879075e-01, 5.19384272e-05,
        1.54648293e-01],
       [1.00657587e-04, 2.68078175e-01, 4.54879075e-01, 5.19384294e-05,
        1.54648293e-01],
       [1.00657587e-04, 2.68078175e-01, 4.54879075e-01, 5.19384280e-05,
        1.54648293e-01],
       [1.00657587e-04, 2.68078175e-01, 4.54879075e-01, 5.19384265e-05,
        1.54648293e-01]]), array([-2546.54743051, -2546.54743051, -2546.54743051, -2546.54743051,
       -2546.54743051, -2546.54743051]))
           fun: -2546.547430514276
       message: 'Optimization terminated successfully.'
          nfev: 799
           nit: 471
        status: 0
       success: True
             x: array([1.00657587e-04, 2.68078175e-01, 4.54879076e-01, 5.19384295e-05,
       1.54648293e-01])

In [19]:
TGARCH_estimated_vol = []
sigma2 = vol*vol
for i in FB_return:
    if i-res_TGARCH.x[3]<0:
        S = 1
    else:
        S = 0
    sigma2 = res_TGARCH.x[0]+res_TGARCH.x[1]*(i-res_TGARCH.x[3])*(i-res_TGARCH.x[3])+res_TGARCH.x[4]*S*(i-res_TGARCH.x[3])*(i-res_TGARCH.x[3])+res_TGARCH.x[2]*sigma2
    TGARCH_estimated_vol.append(sigma2)
TGARCH_estimated_vol.pop
TGARCH_estimated_vol = pd.DataFrame(TGARCH_estimated_vol, index = FB_return.index, columns = ['TGARCH_estimated_vol'])
TGARCH_estimated_vol.mean()

TGARCH_estimated_vol    0.0004
dtype: float64

In [30]:
#np.sqrt(TGARCH_estimated_vol*252).to_pickle('Estimated Volatility/FB_TGARCH_estimated_vol.pkl')

## IGARCH

In [21]:
parameter_IGARCH = np.array([vol,0.66,FB_return_mean])#alfa_0,beta_1,miu


def L_IGARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in FB_return.loc[FB_return.index<pd.to_datetime('2019-11-18')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[2])*(i-parameter_0[2])/(sigma2)/2
        sigma2 = parameter_0[0]+(1-parameter_0[1])*(i-parameter_0[2])*(i-parameter_0[2])+parameter_0[1]*sigma2
    return result

res_IGARCH = minimize(L_IGARCH, parameter_IGARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_IGARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2543.009931
         Iterations: 289
         Function evaluations: 544


 final_simplex: (array([[6.14929746e-05, 5.34975569e-01, 7.77054206e-05],
       [6.14929747e-05, 5.34975568e-01, 7.77054197e-05],
       [6.14929746e-05, 5.34975569e-01, 7.77054163e-05],
       [6.14929744e-05, 5.34975570e-01, 7.77054200e-05]]), array([-2543.00993063, -2543.00993063, -2543.00993063, -2543.00993063]))
           fun: -2543.0099306319757
       message: 'Optimization terminated successfully.'
          nfev: 544
           nit: 289
        status: 0
       success: True
             x: array([6.14929746e-05, 5.34975569e-01, 7.77054206e-05])

In [22]:
IGARCH_estimated_vol = []
sigma2 = vol*vol
for i in FB_return:
    sigma2 = res_IGARCH.x[0]+(1-res_IGARCH.x[1])*(i-res_IGARCH.x[2])*(i-res_IGARCH.x[2])+res_IGARCH.x[1]*sigma2
    IGARCH_estimated_vol.append(sigma2)
IGARCH_estimated_vol.pop
IGARCH_estimated_vol = pd.DataFrame(IGARCH_estimated_vol, index = FB_return.index, columns = ['IGARCH_estimated_vol'])
IGARCH_estimated_vol.mean()

IGARCH_estimated_vol    0.000468
dtype: float64

In [31]:
#np.sqrt(IGARCH_estimated_vol*252).to_pickle('Estimated Volatility/FB_IGARCH_estimated_vol.pkl')

In [24]:
-np.log(-res_ARCH.fun)+3*np.log(975)

12.812727982505997

In [25]:
-np.log(-res_GARCH.fun)+4*np.log(975)

19.68754447184735

In [26]:
-np.log(-res_TGARCH.fun)+5*np.log(975)

26.56969358307158

In [27]:
-np.log(-res_IGARCH.fun)+3*np.log(975)

12.806208742493725